These data come from a survey conducted by INSEE, Budget des Familles, which are protected by the Statistical Secrecy Committee. To reproduce the analysis, one must request access to this data.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('BDF/Csv/C05.csv', sep=";", index_col="IDENT_MEN")

We need to remove the columns starting with 13 (taxes) and 14 (subsidies) which are not consumption. And also the columns total CTOT and pondmen (weight household) which are not useful.

In [3]:
columns_to_delete = df.columns[df.columns.str.startswith('C13') | df.columns.str.startswith('C14')]
df = df.drop(columns=columns_to_delete)
df = df.drop(columns=["CTOT", "pondmen"])

In [4]:
df_prop = df.divide(df.sum(axis=1), axis=0)


In [5]:
df_prop.rename(columns=lambda x: x.replace('C', ''), inplace=True)


We know have the proportion of each consumption area per household.

In [6]:
import eurostat

In [7]:
df_inflation_raw = eurostat.get_data_df(code="PRC_HICP_MANR", filter_pars={
                                    'geo': 'FR', 'startPeriod': "2021-03"})
df_inflation_raw = df_inflation_raw.rename(columns={'geo\TIME_PERIOD': 'geo'})

In [8]:
df_correspondances = pd.read_csv("correspondances.csv", dtype="str", sep=";")

In [9]:
df_correspondances.set_index('colonnes de df_prop', inplace=True)

In [10]:
df_inflation = df_inflation_raw[df_inflation_raw['coicop'].str.startswith('CP')]

In [11]:
df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


C:\Users\mdaversa\AppData\Local\Temp\ipykernel_9776\2914206753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


In [12]:
df_inflation = df_inflation.loc[:, ["coicop", "2023-04"]]


In [13]:
df_inflation = df_inflation.set_index('coicop')

In [14]:
def cell_func(cell_value, col_name):
    # Votre logique ici, vous pouvez utiliser 'col_name' et 'cell_value'
    coicop = df_correspondances.loc[col_name]['colonnes de df_inflation correspondante']
    return df_inflation.loc[coicop]["2023-04"]*cell_value

def col_func(col):
    column_name = col.name
    return col.apply(cell_func, args=(column_name,))


test = df_prop.apply(col_func)

In [15]:
df_inflation_by_household = pd.DataFrame(test.sum(axis=1))

In [16]:
df_inflation_by_household = df_inflation_by_household.rename(columns={0: "inflation"})

# Household

In [17]:
df_menage = pd.read_csv("BDF/Csv/MENAGE.csv", sep=";", encoding='latin1')

C:\Users\mdaversa\AppData\Local\Temp\ipykernel_9776\1221685487.py:1: DtypeWarning: Columns (21,22,23,48,49,421) have mixed types. Specify dtype option on import or set low_memory=False.
  df_menage = pd.read_csv("BDF/Csv/MENAGE.csv", sep=";", encoding='latin1')


In [18]:
df_menage = df_menage.set_index("IDENT_MEN")

In [19]:
df_menage = df_menage.merge(df_inflation_by_household, on ="IDENT_MEN")

C:\Users\mdaversa\AppData\Local\Temp\ipykernel_9776\2296896769.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_menage = df_menage.merge(df_inflation_by_household, on ="IDENT_MEN")


In [20]:
df_menage.describe()

,BS,CATLOGAYP,CATLOGAYNP,NPERS,NCOUPLES,NENFANTS,NACTIFS,NENFACT,TYPMEN5,TYPMEN15,...,NIVIE,QNIVIE1,DNIVIE1,QNIVIE2,DNIVIE2,TUU,CATAEU,TAU,dom,inflation
count,16978.000000,0.0,0.0,16978.000000,16978.000000,16978.000000,16978.000000,16978.000000,16978.000000,16978.000000,...,16978.000000,16978.000000,16978.000000,16978.000000,16978.000000,16978.00000,16975.000000,16975.000000,16978.000000,16978.000000
mean,0.008010,NaN,NaN,2.526799,0.539698,0.928849,1.050300,0.105195,2.655613,28.095948,...,20259.519555,2.728943,4.915067,2.894098,5.271763,4.29285,143.515700,6.356053,0.872188,6.547546
std,0.089144,NaN,NaN,1.498705,0.503960,1.269950,0.892167,0.362276,1.271557,13.283495,...,13494.234006,1.480242,3.055316,1.435290,2.931871,2.77065,75.946362,3.244776,1.575992,2.214463
min,0.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,10.000000,...,-28571.000000,1.000000,1.000000,1.000000,1.000000,0.00000,111.000000,0.000000,0.000000,-6.342854
25%,0.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,...,12057.750000,1.000000,2.000000,2.000000,3.000000,2.00000,111.000000,5.000000,0.000000,5.152089
50%,0.000000,NaN,NaN,2.000000,1.000000,0.000000,1.000000,0.000000,3.000000,32.000000,...,18248.500000,3.000000,5.000000,3.000000,5.000000,5.00000,111.000000,7.000000,0.000000,6.458141
75%,0.000000,NaN,NaN,3.000000,1.000000,2.000000,2.000000,0.000000,4.000000,42.000000,...,25722.750000,4.000000,8.000000,4.000000,8.000000,7.00000,112.000000,9.000000,1.000000,7.817196
max,1.000000,NaN,NaN,14.000000,4.000000,10.000000,9.000000,5.000000,5.000000,53.000000,...,285815.000000,5.000000,10.000000,5.000000,10.000000,8.00000,400.000000,10.000000,5.000000,20.306288


In [21]:
df_menage_filtered = df_menage.loc[:,["AGEPR", "QNIVIE1", "SEXEPR", "TAU", "TYPMEN5", "inflation"]]

In [22]:
df_menage_filtered.head()


,AGEPR,QNIVIE1,SEXEPR,TAU,TYPMEN5,inflation
IDENT_MEN,,,,,,
1,73,4,1,4.0,3,7.106916
2,65,2,2,2.0,1,5.140171
3,40,1,1,7.0,4,7.845245
4,78,2,1,0.0,3,3.212936
5,56,3,1,8.0,4,6.094502


In [23]:
X = df_menage_filtered[['AGEPR', 'QNIVIE1', 'TYPMEN5']]
y = df_menage_filtered['inflation']


In [24]:
df_menage_filtered = df_menage_filtered.dropna() 

In [25]:
import numpy as np
df_menages_filtered = df_menage_filtered.replace([np.inf, -np.inf], np.nan)  # Remplace les infinies par des NaN

In [26]:
import statsmodels.api as sm
X = sm.add_constant(X)

In [27]:
model = sm.OLS(y, X)

In [28]:
results = model.fit()


In [29]:
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:              inflation   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     135.1
Date:                Fri, 09 Jun 2023   Prob (F-statistic):           1.79e-86
Time:                        13:55:50   Log-Likelihood:                -37388.
No. Observations:               16978   AIC:                         7.478e+04
Df Residuals:                   16974   BIC:                         7.481e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1808      0.076     68.540      0.0

In [58]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

# Charger les données dans un DataFrame (supposons que vous les avez déjà chargées)
data = df_menages_filtered

# Transformation de AGEPR en catégories
age_bins = pd.qcut(data["AGEPR"], q=5, labels=False)  # Découpe en quintiles
data["AGEPR_category"] = age_bins

# Conversion de TYPMEN5 en variable catégorielle
encoder = OneHotEncoder(sparse=False, drop="first")
typmen_encoded = encoder.fit_transform(data[["TYPMEN5"]])
typmen_categories = encoder.categories_[0][1:]  # Ignorer la première catégorie
typmen_df = pd.DataFrame(typmen_encoded, columns=[f'typmen_{str(cat)}' for cat in typmen_categories], index=data.index)
data = pd.concat([data, typmen_df], axis=1)

# Conversion de TYPMEN5 en variable catégorielle
encoder2 = OneHotEncoder(sparse=False, drop="first")
agepr_encoded = encoder2.fit_transform(data[["AGEPR_category"]])
agepr_categories = encoder.categories_[0][1:]  # Ignorer la première catégorie
agepr_df = pd.DataFrame(agepr_encoded, columns=[f'agepr_{str(cat)}' for cat in agepr_categories], index=data.index)
data = pd.concat([data, agepr_df], axis=1)


c:\Users\mdaversa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\mdaversa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [62]:
age_bins = pd.cut(df_menage_filtered['AGEPR'], 5, labels=False)
df_menage_filtered['AGEPR_category'] = age_bins
X = df_menage_filtered['AGEPR_category']
X = pd.get_dummies(X, drop_first=True)

In [63]:
y = data["inflation"]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              inflation   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     55.34
Date:                Fri, 09 Jun 2023   Prob (F-statistic):           1.93e-46
Time:                        14:24:48   Log-Likelihood:                -37471.
No. Observations:               16975   AIC:                         7.495e+04
Df Residuals:                   16970   BIC:                         7.499e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.9734      0.048    125.202      0.0

In [ ]:

# Préparation des données pour la régression linéaire
X = data[["AGEPR_category"] + list(typmen_df.columns)]
y = data["inflation"]

# Ajustement du modèle de régression linéaire multiple
model = LinearRegression()
model.fit(X, y)

# Afficher les coefficients de régression
coefficients = pd.DataFrame({"Variable": ["Intercept"] + ["AGEPR_category"] + list(typmen_df.columns),
                            "Coefficient": np.concatenate([[model.intercept_], model.coef_])})
print(coefficients)


In [35]:
from sklearn.metrics import r2_score

# Calcul du coefficient de détermination (R²)
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

# Création d'un tableau de coefficients
coefficients = pd.DataFrame({"Variable": ["Intercept"] + ["AGEPR_category"] + list(typmen_df.columns),
                            "Coefficient": np.concatenate([[model.intercept_], model.coef_])})

# Affichage du tableau de coefficients
print("Tableau des coefficients :")
print(coefficients)

# Affichage du coefficient de détermination (R²)
print("Coefficient de détermination (R²) :", r2)

Tableau des coefficients :
         Variable  Coefficient
0       Intercept     5.778332
1  AGEPR_category     0.212693
2               2     0.300069
3               3     0.452394
4               4     0.627503
5               5     0.492161
Coefficient de détermination (R²) : 0.02343268325971759
